# 데이터 분석 실무
## 파이썬 라이브러리를 활용해서 통계 기법을 적용한 데이터 분석을 배우는 과정입니다.

#### 이번 시간은 제주도 관광데이터를 활용해서 분석을 해보는 시간입니다.
* 먼저, 데이터를 불러오고 살펴보겠습니다.(jeju_2018.csv, encoding 'cp949')
* 내국인 관광객 지역, 업종, 성별, 연령대별 카드 이용 데이터
* https://www.jejudatahub.net/data/view/data/597

In [ ]:
# 필요 라이브러리 import
import pandas as pd

In [ ]:
# 데이터 불러오기
df_jeju = pd.read_csv('jeju_2018.csv', encoding='cp949')
df_jeju

##### 변수 요약
* sido: 시도
* sigungu: 시군구
* cityGubun: 도심구분
* marketType: 업종
* userType: 관광객구분
* ageGroup: 연령대
* gender: 성별
* dtYearMonth: 년월
* userCount: 고객수
* useCount: 사용횟수
* useCost: 사용금액

#### 두 집단 간 차이 분석 : t-test

##### 가설설정
* 가설 1: 월별 남성과 여성의 이용 횟수의 차이가 있는지 여부를 검정한다.
* 가설 2: 월별 20대와 30대의 이용 비용의 차이가 있는지 여부를 검정한다.
* 가설 3: 월별 도심과 읍면 지역의 이용자 수의 차이가 있는지 여부를 검정한다.

In [ ]:
# 필요 라이브러리 import
import pandas as pd
from scipy import stats

* 가설 1 검정

In [ ]:
# 가설 1: 남성과 여성의 이용 횟수 비교
## 데이터 생성
male_data = df_jeju[df_jeju['gender'] == '남'][['useCount','dtYearMonth']].groupby('dtYearMonth').sum()
female_data = df_jeju[df_jeju['gender'] == '여'][['useCount','dtYearMonth']].groupby('dtYearMonth').sum()

In [ ]:
## 검정통계량 계산
t_statistic, p_value = stats.ttest_ind(male_data, female_data)

print(p_value)

In [ ]:
## 결과 도출
if p_value < 0.05:
    print("남성과 여성의 이용 횟수는 통계적으로 유의미한 차이가 있습니다.")
else:
    print("남성과 여성의 이용 횟수는 통계적으로 유의미한 차이가 없습니다.")

* 가설 2 검정

In [ ]:
# 가설 2: 20대와 30대의 이용 비용 비교
## 데이터 생성
age_20s_data = df_jeju[df_jeju['ageGroup'] == '20대'][['useCost','dtYearMonth']].groupby('dtYearMonth').sum()
age_30s_data = df_jeju[df_jeju['ageGroup'] == '30대'][['useCost','dtYearMonth']].groupby('dtYearMonth').sum()

In [ ]:
## 검정통계량 계산
t_statistic, p_value = stats.ttest_ind(age_20s_data, age_30s_data)

print(p_value)

In [ ]:
## 결과 도출
if p_value < 0.05:
    print("20대와 30대의 이용 비용은 통계적으로 유의미한 차이가 있습니다.")
else:
    print("20대와 30대의 이용 비용은 통계적으로 유의미한 차이가 없습니다.")

* 가설 3 검정

In [ ]:
# 가설 3: 도심과 읍면 지역의 이용자 수 비교
## 데이터 생성
urban_data = df_jeju[df_jeju['cityGubun'] == '도심'][['userCount','dtYearMonth']].groupby('dtYearMonth').sum()
rural_data = df_jeju[df_jeju['cityGubun'] == '읍면'][['userCount','dtYearMonth']].groupby('dtYearMonth').sum()

In [ ]:
## 검정통계량 계산
t_statistic, p_value = stats.ttest_ind(urban_data, rural_data)

print(p_value)

In [ ]:
## 결과 도출
if p_value < 0.05:
    print("도심과 읍면 지역의 이용자 수는 통계적으로 유의미한 차이가 있습니다.")
else:
    print("도심과 읍면 지역의 이용자 수는 통계적으로 유의미한 차이가 없습니다.")

#### 세 집단 이상 차이 분석 : ANOVA

##### 가설설정
* 가설 1: 'marketType'에 따라 'userCount'가 유의미하게 다를까요?
* 가설 2: 'ageGroup'에 따라 'userCount'가 유의미하게 다를까요?
* 가설 3: 'marketType'과 'ageGroup'의 상호작용이 'userCount'에 유의미한 영향을 미치나요?

* 가설 1 검정

In [ ]:
# 필요 라이브러리 import
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# 가설 1: marketType에 따른 userCount 차이 분산분석
## 데이터 생성
market_data = df_jeju[['userCount','marketType','dtYearMonth']].groupby(['marketType','dtYearMonth']).sum().reset_index()

In [ ]:
## 검정통계량 계산
model_1 = ols('userCount ~ marketType', data=market_data).fit()
anova_table_1 = sm.stats.anova_lm(model_1)

In [ ]:
## 결과 도출
print("=== 가설 1: marketType에 따른 userCount 분산분석 결과 ===")
print(anova_table_1)

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# DataFrame의 'marketType'과 'userCount' 열을 사용하여 사후 검정 수행
posthoc = pairwise_tukeyhsd(market_data['userCount'], market_data['marketType'])

# 사후 검정 결과 출력
print(posthoc)

* 가설 2 검정

In [ ]:
# 필요 라이브러리 import
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# 가설 1: ageGroup에 따른 userCount 차이 분산분석
## 데이터 생성
age_data = df_jeju[['userCount','ageGroup','dtYearMonth']].groupby(['ageGroup','dtYearMonth']).sum().reset_index()

In [ ]:
## 검정통계량 계산
model_2 = ols('userCount ~ ageGroup', data=age_data).fit()
anova_table_2 = sm.stats.anova_lm(model_2)

In [ ]:
## 결과 도출
print("\n=== 가설 2: ageGroup에 따른 userCount 분산분석 결과 ===")
print(anova_table_2)

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# DataFrame의 'marketType'과 'userCount' 열을 사용하여 사후 검정 수행
posthoc = pairwise_tukeyhsd(age_data['userCount'], age_data['ageGroup'])

# 사후 검정 결과 출력
print(posthoc)

* 가설 3 검정

In [ ]:
# 필요 라이브러리 import
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# 가설 1: ageGroup에 따른 userCount 차이 분산분석
## 데이터 생성
merge_data = df_jeju[['userCount','marketType','ageGroup','dtYearMonth']].groupby(['marketType','ageGroup','dtYearMonth']).sum().reset_index()

In [ ]:
## 검정통계량 계산
model_3 = ols('userCount ~ marketType * ageGroup', data=merge_data).fit()
anova_table_3 = sm.stats.anova_lm(model_3)

In [ ]:
## 결과 도출
print("\n=== 가설 3: marketType과 ageGroup의 상호작용이 userCount에 미치는 영향 분산분석 결과 ===")
print(anova_table_3)

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 상호작용항 만들기
merge_data['combi'] = merge_data['marketType'] + " / " + merge_data['ageGroup']

# 사후 검정 결과 계산
posthoc = pairwise_tukeyhsd(merge_data['userCount'], merge_data['combi'])

# 사후 검정 결과 출력
pd.set_option('display.max_rows', None)
posthoc._results_table

#### 상관분석

In [ ]:
# 필요 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터프레임의 변수 간 상관계수 계산
correlation_matrix = df_jeju.corr(numeric_only=True)

In [ ]:
# 상관계수 출력
print(correlation_matrix)

In [ ]:
# 상관계수를 히트맵으로 시각화
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

#### 회귀분석
* 종속변수는 useCost
* 독립변수는 marketType,	userType,	ageGroup,	gender

In [ ]:
import pandas as pd
import statsmodels.api as sm

# 범주형 변수 변환

categorical_cols = ['marketType', 'userType', 'ageGroup', 'gender']
data_encoded = pd.get_dummies(df_jeju[['marketType', 'userType', 'ageGroup', 'gender','useCost']], columns=categorical_cols, drop_first=True)

# 독립변수/종속변수 설정
y = data_encoded['useCost']
X = data_encoded.drop('useCost', axis=1)

# 상수항 추가
X = sm.add_constant(X)

# Float로 형변환
X = X.astype(float)

# 회귀식 도출
model = sm.OLS(y, X).fit()

# 결과 출력
print(model.summary())

In [ ]:
X.dtypes

In [ ]:
# Extract the regression results
results = pd.DataFrame({'coef': model.params, 'std_err': model.bse, 't_value': model.tvalues, 'p_value': model.pvalues})

# Print the results DataFrame
print(results)